The set of PSD matrices
$$\mathcal{S}^n_+ := \{ X \in \mathcal{S} : z^T X z \geq 0 \ \forall z \in \mathbb{R}^n \},$$
where $\mathcal{S}^n$ is the set of symmetric n by n matrices, forms a convex cone.

If symmetric matrix $Q$ has eigenvalues $\lambda_1, \lambda_2 \ldots \lambda_n$, then $tI-Q$ has eigenvalues $t-\lambda_1, t-\lambda_2 \ldots t-\lambda_n$.  $tI-Q$ is PSD exactly when all its eignevalues are non-negative, which is true only when $t \geq \lambda_1$ its largest eigenvalue.  Thus, we can model the problem of finding the largest eigenvalue of a symmetric matrix $Q$ with the following optimization problem 

$$\lambda_1 = \min t : tI - Q \succeq 0$$

In [3]:
# This one will only work with SCS optimizer. 
using JuMP, LinearAlgebra, SCS 

A = [3 2 4; 2 0 2; 4 2 3]
I = Matrix{Float64}(LinearAlgebra.I, 3, 3)
model = Model(SCS.Optimizer)
set_silent(model)
@variable(model, t)
@objective(model, Min, t)
@constraint(model, t .* I - A in PSDCone())
optimize!(model)
if termination_status(model) == MOI.OPTIMAL && primal_status(model) == MOI.FEASIBLE_POINT
    println("The model is solved and feasible.")
else
    println("The model is not solved or not feasible.")
end
objective_value(model)

The model is solved and feasible.


8.000003377698677

Let's see an SDP where we minimize the amount added to the diagonal of a symmetric (non-PSD) matrix in order to make it PSD 

In [4]:
using LinearAlgebra 
Q = [2 4 -2 -1; 4 -1 3 0; -2 3 0 0; -1 0 0 3]
eig = eigen(Q)
print("Min eigenvalue is ", minimum(eig.values))

# Now lets find the smallest amount to Q to make it PSD 
model = Model(SCS.Optimizer)
@variable(model, X[1:4, 1:4])
@objective(model, Min, sum(X[j,j] for j in 1:4))

for i in 1:4
    for j in 1:4
        if i != j
            @constraint(model, X[i,j] == 0)
        end 
    end 
end 

@constraint(model, (X + Q) in PSDCone())

#print(model)

optimize!(model)
if termination_status(model) == MOI.OPTIMAL && primal_status(model) == MOI.FEASIBLE_POINT
    println("The model is solved and feasible.")
else
    println("The model is not solved or not feasible.")
end
print(objective_value(model))
value.(X)

Min eigenvalue is -5.980990681956462------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 16, constraints m: 28
cones: 	  z: primal zero / dual free vars: 18
	  s: psd vars: 10, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 34, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.13e+01  1.00e+00  1.22e+02 -6.03e+01  1.00e-01  1.20e-04 
    75| 1.57e-05  2.96e-06  2.70e-05  1.60e+01  1.00e-01  4.59

4×4 Matrix{Float64}:
  5.00001      -3.35548e-9   -4.03379e-10  -9.22264e-9
 -1.45476e-9    8.00001      -6.79211e-10   9.55323e-9
 -1.74883e-10  -2.94469e-10   5.00001      -1.18989e-8
 -3.99843e-9    4.14176e-9   -5.15873e-9   -1.99996